 # Análise de Acidentes de Trânsito em Porto Alegre 🚦

Este projeto tem como objetivo realizar uma análise exploratória dos acidentes de Trânsito
em Porto Alegre, identificando padrões temporais e espaciais que possam apoiar estratégias de prevenção.

#### Fontes dos Dados 
https://dadosabertos.poa.br/dataset/acidentes-de-transito-acidentes


#### Objetivos principais :
- Explorar os dados de acidentes em Porto Alegre
- Identificar os período e locais mais críticos
- Gerar insight que possam apoiar políticas públicas e conscientização.

## 1.Importação e Visualização inicial  📥 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Carregando o dataframe que utilizaremos na análise.
df = pd.read_csv('E:\Projeto_anaconda\dadosACD\cat_acidentes.csv', sep = ';')

In [ ]:
df

In [ ]:
df.info()

# 2.Limpeza e Preparo de Dados
- Limpeza de valores NaN
- Padronização das colunas
  

In [ ]:
df = df.rename(columns={'noite_dia': 'periodo'})
# Mudei o nome da coluna "noite_dia" para "periodo", para que seja mais fácil de se manipular

In [ ]:
df

In [ ]:
# Nessa parte limpamos os valores nulos que estavam nas colunas "longitude" e "latitude"
df = df.dropna(subset=['longitude', 'latitude'], how = 'any' )
df

Agora com os dados preparados e organizados podemos começar a nossa análise. Irei começar fazendo uma Análise Geográficas, usando mapas como de calor e de clusters para identificar concentrações de acidentes em Porto Alegre.

# 3.Análise Geográfica 🗺️

In [ ]:

import folium
from folium.plugins import HeatMap, MarkerCluster

mapa = folium.Map(location=[-30.1, -51.15], zoom_start=11)
coordenadas = list(zip(df.latitude, df.longitude))
HeatMap(coordenadas, radius=9, blur=10).add_to(mapa)
mapa



No mapa de calor, conseguimos ver que a região central de Porto Alegre é a que concentra mais acidentes, já que aparece como a área mais intensa do mapa.

Para entender melhor essa concentração, vamos usar um mapa de clusters, que ajuda a visualizar de forma mais clara e detalhada onde os acidentes realmente acontecem.

In [ ]:
from folium.plugins import MarkerCluster

mapa = folium.Map(location=[-30.1,-51.15], zoom_start=11)
mapa_cluster= MarkerCluster(coordenadas)
mapa.add_child(mapa_cluster)
              
mapa



Agora já sabemos quantos acidentes aconteceram no centro de Porto Alegre e nas regiões ao redor. Mas surge uma nova pergunta: em quais anos esses acidentes foram registrados?

## 4.Frequência de Acidentes
Primeiro, vamos examinar o **DataFrame** para identificar de quais anos os dados estão sendo coletados. Essa etapa nos permite ter uma visão geral do período abrangido pelos registros de acidentes.


In [ ]:
df.data

In [ ]:
df['data'] = pd.to_datetime(df['data'], errors= 'coerce')
df_ano = df['data'].dt.year.value_counts()
df_ano

O resultado mostra os anos que os acidentes foram registrados, permitindo identificar os períodos com maior registro de ocorrências.

In [ ]:
# Gráfico dos Registros por ano de Acidentes
df_ano = df['data'].dt.year.value_counts().sort_index()

anos = df_ano.index
valores = df_ano.values
cores = plt.cm.Blues(np.linspace(0.4,1, len(anos)))

plt.figure(figsize=(10,6))
plt.bar(anos, valores, color=cores, edgecolor = 'black')
plt.title('Frequência de Registro por Ano', fontsize=14)
plt.xlabel("Ano", fontsize=12)
plt.ylabel("Frequência", fontsize=12)

for i, v in enumerate(valores):
    plt.text(anos[i], v + 1, str(v), ha='center', fontsize=10)


plt.show()

Agora sabemos quais anos estão registrados no **DataFrame** e podemos ter uma noção da quantidade de acidentes por ano.  
Observa-se que **2023** foi o ano com o maior número de registros de acidentes de trânsito, indicando um aumento significativo nesse período.

In [ ]:
tabela = df['dia_sem'].value_counts().to_frame(name='frequencia')
tabela.reset_index(inplace=True)
tabela.columns = ['dia_sem', 'frequencia']
tabela

# Esta tabela nos permite visualizar quais dias da semana apresentam maior, frequência de acidentes.

Observa-se que **sexta-feira** é o dia da semana com maior ocorrência de acidentes nos últimos anos.  
Essa informação pode ser útil para direcionar campanhas de conscientização e medidas de prevenção de acidentes nos dias de maior risco.

In [ ]:
dia_sexta = 'SEXTA-FEIRA'
df_filtrado = df[df['dia_sem'] == dia_sexta]

In [ ]:
tabela_periodo = df_filtrado['periodo'].value_counts().to_frame(name = 'quantidade')
tabela_periodo.reset_index(inplace=True)
tabela_periodo.columns = ['periodo', 'quantidade']

tabela_periodo
# A tabela permite comparar a quantidade de acidentes durante o dia e à noite.

Agora que identificamos que a quantidade que mais ocorrem acidentes é no **período do dia**, vamos aprofundar a análise criando um gráfico para visualizar **quais horários apresentam maior frequência de acidentes**.  
Isso nos ajudará a entender melhor os momentos de maior risco no trânsito.

In [ ]:
df['hora'].isna().sum
# Checando se a algum valor nulo na coluna "hora" do data frame para não nos prejudicar.

Existe alguns valores nulos na coluna e iremos retirar-los 

In [ ]:
df_hora = df.dropna(subset=['hora'],how = 'any')
df_hora

In [ ]:
df_hora['hora'].isna().sum()
# Agora a coluna está totalmente limpa de valores nulos

In [ ]:
df_sexta = df[df['dia_sem'] == 'SEXTA-FEIRA']
df_sexta['hora'] = pd.to_datetime(df_sexta['hora'], errors = 'coerce').dt.time
df_sexta['hora_do_dia'] = df_sexta['hora'].apply(lambda x: x.hour)

In [ ]:
bins = list(range(0, 25, 2))
labels = [f'{i}-{i+1}h' for i in bins[:-1]]
df_sexta['intervalo_hora'] = pd.cut(df_sexta['hora_do_dia'], bins=bins, labels=labels, right=False)

acidentes_hora = df_sexta['intervalo_hora'].value_counts().sort_index()
acidentes_hora

# Para entender melhor quantos os acidentes ocorrem durante o dia, dividimos em intervalos de 2 horas e contamos a quantidade de acidentes.

Podemos ver que os intervalos com maior ocorrência de acidentes são ao início do dia **8-9h** e no período da tarde **16-17h**, indicando que os horários de pico do tráfego correspondem a momentos de maior risco de acidentes.
Intervalos da madrugada, como 2-3h e 4-5h, apresentam menor número de acidentes.

In [ ]:
acidentes_hora = acidentes_hora.sort_index()

norm = acidentes_hora.values / acidentes_hora.values.max()
cores = plt.cm.Blues(norm)

plt.figure(figsize=(12,6))
plt.bar(acidentes_hora.index.astype(str), acidentes_hora.values, color=cores)
plt.title("Acidentes de Trânsito por Intervalo de Hora (Sexta-feira)", fontsize=16)
plt.xlabel("Intervalo de Hora", fontsize=12)
plt.ylabel("Quantidade de Acidentes", fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

## 5.Relação entre  variáveis
Nessa etapa,analisamos como os acidentes se distribuem ao longo da semana e em diferentes períodos do dia.

In [ ]:
dias_ordem = ['SEGUNDA-FEIRA','TERÇA-FEIRA','QUARTA-FEIRA','QUINTA-FEIRA','SEXTA-FEIRA','SÁBADO','DOMINGO' ]
df['dia_sem'] = pd.Categorical(df['dia_sem'], categories=dias_ordem, ordered=True)

In [ ]:
# Criei uma tabela para nos mostrar os dias da semana e quantidade de acidentes

tabela_dia_periodo = pd.crosstab(df['dia_sem'], df['periodo'])
tabela_dia_periodo

In [ ]:
tabela_dia_periodo = pd.crosstab(df['dia_sem'], df['periodo'])

ax = tabela_dia_periodo.plot(kind='bar', figsize=(10,6))
plt.title('Acidentes por Dia da semana e Período do dia')
plt.xlabel('Dia da Semana')
plt.ylabel('Quantidade de Acidentes')
plt.legend(title='Periodo do Dia')

for container in ax.containers:  
    ax.bar_label(container, fmt='%d')

plt.show()

Observando o gráfico, podemos identificar que o período do **dia** são mais frequentes de acidentes. Com por exemplo, notamos que há uma maior concentração de acidentes nos **finais de semana** ou durante o **período da manhã**, pois o fluxo de carros aumenta nesses períodos. Essas informações pode auxiliar no planejamento de campanhas de segurança viária e fiscalização.

# Acidentes por hora e dia da semana
Nesta etapa, analisamos a distribuição dos acidentes ao longo do **dia** em cada **dia da semana**, para identificar quais horários apresentam maior ocorrência de acidentes.

In [ ]:
print(df.columns)

In [ ]:
print(df['dia_sem'].unique())

In [ ]:
print(df['periodo'].unique())

In [ ]:
dias_ordem = ['SEGUNDA-FEIRA','TERÇA-FEIRA','QUARTA-FEIRA','QUINTA-FEIRA','SEXTA-FEIRA','SÁBADO','DOMINGO' ]
df['dia_sem'] = pd.Categorical(df['dia_sem'], categories=dias_ordem, ordered=True)

In [ ]:
# Criando uma tabela cruzada entre hora e dia da semana
tabela_dia_periodo = pd.crosstab(df['dia_sem'],df['periodo'])
tabela_dia_periodo

In [ ]:
df['hora'] = pd.to_datetime(df['hora'], errors='coerce')
df['hora_int'] = df['hora'].dt.hour

tabela_hora_dia = pd.crosstab(df['hora_int'], df['dia_sem'])

plt.figure(figsize=(12,6))
sns.heatmap(
    tabela_hora_dia,
    cmap='Blues',
    annot = True,
    fmt='d',
    cbar_kws={'label': 'Quantidade de Acidentes'} 
)

plt.title('Distribuição de Acidentes por Hora do Dia e Dia da semana')
plt.xlabel('Dia da semana')
plt.ylabel('Hora do Dia')
plt.show()
                

O heatmap permite visualizar os picos de acidentes em horários específicos. Podemos ver claramente quais horários são mais críticos em cada dia da semana, com destaque para 07:00 e 08:00 da manhã e 18:00 às 19:00 da noite, que concentram maior número de acidentes. Essa informação é fundamental para planejar campanhas de conscientização e estratégias de fiscalização.

# Conclusão

Ao longo deste projeto, realizamos uma análise exploratória dos acidentes, observando sua distribuição por **dia da semana**, **período do dia** e **hora do dia**. Alguns pontos importantes foram identificados:

- Os acidentes apresentam maior ocorrência em determinados dias e horários, com destaque para **07:00 e 08:00 da manhã** e **18:00 às 19:00 da noite**, indicando horários críticos, possivelmente relacionados ao fluxo intenso de trânsito para ida e volta de trabalho.
- A análise por dia da semana mostrou padrões de maior incidência nos dias de maior movimento, especialmente **sextas-feiras** e períodos próximos ao final da semana.
- A distribuição por período e hora permite direcionar ações de conscientização, fiscalização e políticas públicas para reduzir o número de acidentes.

Esses insights são fundamentais para **planejar estratégias de prevenção**, identificar áreas e horários críticos, e auxiliar na tomada de decisões por órgãos de trânsito e gestores urbanos.
